In [4]:
from pyspark.sql.types import *
from pyspark.sql import Row

# Introduction
We are going to use the dating profiles dataset for this tutorial. More information on this dataset can be found [here](https://sites.google.com/a/insightdatascience.com/spark-lab/s3-data/dating-profiles).

# Loading in the data
First, read in the data into an RDD. Here, we are using 16 partitions for our RDD.

In [1]:
# Read in raw ratings data (fromUserId, toUserId, rating)
ratingsCsvRDD = sc.textFile("s3n://insight-spark-after-dark/ratings.csv.gz").repartition(16)

Reformat the data into a JSON format by applying the function rec_tup (which converts the data into a JSON format) to the RDD using the .map and .toDF() transformation steps.

In [5]:
# Convert raw ratings RDD to Json RDD
def rec_tup(rating):
    tokens = rating.split(",")
    return Row(fromUserId=int(tokens[0]), toUserId=int(tokens[1]), rating=int(tokens[2]))

ratingsJson_DF = ratingsCsvRDD.map(rec_tup).toDF()
ratingsJson_DF.take(5)

[Row(fromUserId=1, rating=8, toUserId=2145),
 Row(fromUserId=1, rating=7, toUserId=11747),
 Row(fromUserId=1, rating=4, toUserId=18878),
 Row(fromUserId=1, rating=4, toUserId=23499),
 Row(fromUserId=1, rating=10, toUserId=35647)]

As you might remember, Spark does a Lazy evaluation which means each transformed RDD may be recomputed each time you run an action on it. We'll use the .persist method (this is actually also a transformation) to keep the result (transformed RDD) on the cluster for quick future access.

**Note**: persist is a transformation, it will only run when a future action is called. One result of this is that if you run an action twice, you will persist twice (and right now in Spark you will lose the pointer to the first persist).

In [6]:
# Cache the SchemaRDD as we'll be using this heavily moving forward
ratingsJson_DF.persist(StorageLevel.MEMORY_AND_DISK_SER)

DataFrame[fromUserId: bigint, rating: bigint, toUserId: bigint]

Quickly check the schema of the dataframe. 
To write SparkSQL, we need to create a table object from our dataframe which we can use to run SparkSQL commands. The transformation registerTempTable() does this and we call our table 'ratingsJsonTable'.

In [8]:
# Describe the SchemaRDD inferred from the JSON
ratingsJson_DF.printSchema()

ratingsJson_DF.registerTempTable("ratingsJsonTable")

root
 |-- fromUserId: long (nullable = true)
 |-- rating: long (nullable = true)
 |-- toUserId: long (nullable = true)



# On to SparkSQL

SparkSQL is an abstract API that lets you use regular SQL commands to query your Spark cluster.

Now that we have created a table of our data we can call SQL commands. 

**Note:** .sql is a transformation and we need to call .collect() to execute the job.

In [14]:
# Details of a table
sqlContext.sql("DESCRIBE ratingsJsonTable").collect()

[Row(col_name=u'fromUserId', data_type=u'bigint', comment=u''),
 Row(col_name=u'rating', data_type=u'bigint', comment=u''),
 Row(col_name=u'toUserId', data_type=u'bigint', comment=u'')]

In [26]:
# Show the top 10 most-active users who are giving out ratings
mostActiveUsersSchemaRDD = sqlContext.sql("""
    SELECT fromUserId, count(*) AS num_rated
    FROM ratingsJsonTable 
    GROUP BY fromUserId
    ORDER BY num_rated DESC
    LIMIT 10
    """)

In [27]:
mostActiveUsersSchemaRDD.collect()

[Row(fromUserId=90280, num_rated=25042),
 Row(fromUserId=56792, num_rated=21599),
 Row(fromUserId=33639, num_rated=19908),
 Row(fromUserId=61436, num_rated=18810),
 Row(fromUserId=72351, num_rated=18443),
 Row(fromUserId=127227, num_rated=18342),
 Row(fromUserId=58765, num_rated=18197),
 Row(fromUserId=76082, num_rated=18019),
 Row(fromUserId=108318, num_rated=17755),
 Row(fromUserId=131976, num_rated=17560)]

# Next Steps

Here are some further questions to get to grips with SparkSQL.

## Which user has the biggest variance in ratings of their profile? 

## How many users have rated each other with a rating > 5?

We can also sample the dataframe with the .sample() transformation

In [29]:
ratingsJson_DF.sample(False, 0.1, 20).collect()

[Row(fromUserId=1, rating=8, toUserId=18444),
 Row(fromUserId=1, rating=8, toUserId=124142),
 Row(fromUserId=1, rating=2, toUserId=144414),
 Row(fromUserId=1, rating=7, toUserId=201108),
 Row(fromUserId=2, rating=6, toUserId=166623),
 Row(fromUserId=4, rating=3, toUserId=56037),
 Row(fromUserId=9, rating=10, toUserId=88),
 Row(fromUserId=9, rating=8, toUserId=11637),
 Row(fromUserId=9, rating=10, toUserId=15641),
 Row(fromUserId=9, rating=8, toUserId=17171),
 Row(fromUserId=9, rating=9, toUserId=30770),
 Row(fromUserId=9, rating=9, toUserId=42994),
 Row(fromUserId=9, rating=6, toUserId=99867),
 Row(fromUserId=9, rating=9, toUserId=101562),
 Row(fromUserId=9, rating=8, toUserId=109904),
 Row(fromUserId=9, rating=5, toUserId=115809),
 Row(fromUserId=9, rating=8, toUserId=116679),
 Row(fromUserId=9, rating=6, toUserId=124129),
 Row(fromUserId=9, rating=10, toUserId=131867),
 Row(fromUserId=9, rating=10, toUserId=134347),
 Row(fromUserId=9, rating=9, toUserId=137745),
 Row(fromUserId=9, ra